In [0]:
stage_table_name = "incremental_load.default.orders_stage"
target_table_name = "incremental_load.default.orders_target"


In [0]:
# Read the data from the stage table
stage_df = spark.read.table(stage_table_name)

In [0]:
try:
    if not spark.catalog.tableExists(target_table_name):
        stage_df.write.format("delta").saveAsTable(target_table_name)
    else:
        from delta.tables import DeltaTable
        target_table = DeltaTable.forName(spark, target_table_name)
        merge_condition = "stage.tracking_num = target.tracking_num"
        
        # Properly chained merge operation
        (target_table.alias("target")
         .merge(stage_df.alias("stage"), merge_condition)
         .whenMatchedUpdateAll()
         .whenNotMatchedInsertAll()
         .execute())
except Exception as e:
    print("Error:", e)